In [1]:
import os
sep_local = os.path.sep
sep_local

In [2]:
import sys
sys.path.append('..'+sep_local+'..')

In [3]:
#done in case of linux
#import os
#os.chdir("/home/azeghost/git/Generative_Models")

In [4]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [5]:
import tensorflow as tf

In [6]:
tf.__version__

'2.0.0'

In [7]:
dataset_name = 'mnist'

In [8]:
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

ERROR:absl:Failed to construct dataset mnist


NotFoundError: NewRandomAccessFile failed to Create/Open: C:\Users\kkh2bp\AppData\Local\Temp\tmpbbwjn375tfds\dataset_info.json : The system cannot find the file specified.
; No such file or directory

In [ ]:
intermediate_dim = 20
inputs_shape=(28, 28, 1) # image shape
batch_size = 100
latent_dim = 50

In [ ]:
TRAIN_BUF = 600
TEST_BUF = 100


In [ ]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)


In [ ]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'][0].numpy().max())
    break

In [ ]:
_instance_scale

In [ ]:
enc_lays2 = [
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(latent_dim)
]

dec_lays2 = [
    tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=3, strides=(1, 1), padding="SAME")
]

In [ ]:
enc_mean_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

enc_var_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu')]

dec_lays = [tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=2*intermediate_dim, activation='relu')]

## First Time

In [ ]:
#from utils.data_and_files.file_utils import make_random_string
#from time import gmtime, strftime

#model_name = 'AE_' + make_random_string(5) + strftime("%a_%d_%b_%Y_%H_%M", gmtime())
#print(model_name)

In [ ]:
model_name = dataset_name+'AE'

In [ ]:
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [ ]:
import os

In [ ]:
os.path.join(recoding_dir, model_name)

In [ ]:
pwd

In [ ]:
from training.traditional.autoencoders.AE import AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_mean_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [ ]:
_restore

In [ ]:
ae = AE( 
    model_name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    variables_params=variables_params, 
    restore=None
    )

In [ ]:
import numpy as np
n_generated = 50
random_latent = np.random.normal(size=[n_generated, latent_dim])
#np.save(file='random_latent.npy', arr=random_latent)
#random_latent = tf.constant(np.load(file='..\\data\\random_latent.npy'))

In [ ]:
ae.fit(
    train_dataset=train_ds, 
    test_dataset=test_ds,
    instance_names=['image'],
    epochs=10,
    learning_rate=1e-3,
    random_latent=random_latent,
    recoding_dir=recoding_dir,
    gray_plot=True,
    generate_epoch=5,
    save_epoch=5,
    metric_epoch=10,
    gt_data=None
)

In [ ]:
#from ground_truth_datasets.datasets import load

In [ ]:
for batch in train_ds:
    image = batch['image'].numpy()[0]
    break

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image.reshape(28,28))

In [ ]:
reconst = ae.decode(ae.encode(image))

In [ ]:
plt.imshow(reconst.numpy().reshape(28,28))